In [1]:
import rasterio, os, shutil, sys
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal, ogr, osr

# 绘图
def draw(self):
    if self.nodata is not None:
        no_data_value = self.nodata
    else:
        no_data_value = np.copy(self.dem_array[0, 0])
    self.dem_array[self.dem_array == no_data_value] = 0
    plt.figure(figsize=(7, 7))
    plt.imshow(self.dem_array, cmap='terrain')
    plt.show()

# 栅格转矢量
def polygonize_raster(self, inputfile, dst_filename):
    # 输入栅格
    inraster = gdal.Open(inputfile)
    inband = inraster.GetRasterBand(1)
    maskband=inband.GetMaskBand()
    prj = osr.SpatialReference()
    prj.ImportFromWkt(inraster.GetProjection())
    # 输出矢量参数设置
    drv = ogr.GetDriverByName('ESRI Shapefile')
    if os.path.exists(dst_filename):
        drv.DeleteDataSource(dst_filename)
    dst_ds = drv.CreateDataSource(dst_filename)
    dst_layer = dst_ds.CreateLayer('out', srs=prj, geom_type=ogr.wkbMultiPolygon)
    dst_fieldname = 'value'
    fd = ogr.FieldDefn(dst_fieldname, ogr.OFTInteger)
    dst_layer.CreateField(fd)
    dst_field = 0
    options=[]
    # Polygonize 函数：输入栅格图像波段\掩码图像波段、矢量化后的矢量图层、需要将DN值写入矢量字段的索引、算法选项、进度条回调函数、进度条参数
    gdal.Polygonize(inband, maskband, dst_layer,dst_field, options)